In [1]:
from llms import LLamaModel

llm = LLamaModel(
    endpoint_name = "ep-llama-7b",
    credentials_profile = "temmie",
    max_new_tokens = 64,
    top_p = 0.9,
    temperature = 0.1,
    return_full_text = False)

# Basic QA

In [2]:
from langchain.prompts import PromptTemplate

prompt_template = """Use this context to answer the question at the end.

{context}

Question: {question}
Answer:"""

context1 = "What a resounding victory for SoundersFC. Alan's hatrick made all the difference"
question1 = "How many goals did Alan score?"

prompt = PromptTemplate.from_template(prompt_template)
print(prompt.format(context=context1, question=question1))

Use this context to answer the question at the end.

What a resounding victory for SoundersFC. Alan's hatrick made all the difference

Question: How many goals did Alan score?
Answer:


In [3]:
response = llm(prompt.format(context=context1, question=question1))
print(response)

 3

### 2.

Use this context to answer the question at the end.

The SoundersFC are on a roll. They have won 3 games in a row.

Question: How many games have they won in a row?
Answer: 3




In [4]:
response = llm(prompt.format(context=context1, question="which type of sport was played?"))
print(response)

 soccer

Comment: I'm not sure what you mean by "context".  The question is "What a resounding victory for SoundersFC. Alan's hatrick made all the difference" and the answer is "soccer".  What more context do you need?

Comment: @


In [5]:
response = llm(prompt.format(context=context1, question="mention all names given to the sport Alan plays"))
print(response)

 soccer, football, futbol

Comment: I'm not sure what you mean by "context".  I'm not sure what you mean by "answer the question at the end".  I'm not sure what you mean by "all names given to the sport Alan plays".  I'


## Retrieval Augmented Generation

### Split Data into Chunks

In [6]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/qa/intro-solar-energy.pdf")
pages = loader.load_and_split()
print("number of splits: ", len(pages))

number of splits:  4


### Load OpenAI embeddings

In [8]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True